In [1]:
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt

In [2]:
import sys
if "../src/" not in sys.path:
    sys.path.insert(0,"../src/")
from pbcpy.grid import DirectGrid, ReciprocalGrid
from pbcpy.field import DirectField, ReciprocalField
from pbcpy.functionals import *

%matplotlib inline
%load_ext autoreload
%autoreload 2
#
from pbcpy.formats.qepp import PP
from pbcpy.formats.xsf import XSF

In [3]:
mol = PP(filepp='Al_fde_rho.pp').read()
rho_of_r = mol.field
grid=mol.cell

In [4]:
precision = 1.0e-8
eta=0.05
sq_eta = eta**2

# Third term

$$
\sum_{ix,iy,iz=-N_x,-N_y,-N_z}^{N_x,N_y,N_z} \sum_{i,j}
$$

In [5]:
grid.lattice

array([[ 3.82291596,  3.82291596,  0.        ],
       [ 3.82291596,  0.        ,  7.64583193],
       [ 0.        ,  3.82291596,  7.64583193]])

In [43]:
L=eta*np.sqrt(np.einsum('ij->j',grid.lattice**2))
N=np.array(sp.erfcinv(precision)/L,dtype=np.int)
sum = np.float(0.0)
for ix in np.arange(-N[0]+1,N[0]):
    for iy in np.arange(-N[1]+1,N[1]):
        for iz in np.arange(-N[2]+1,N[2]):
            R=np.einsum('j,ij->i',np.array([ix,iy,iz],dtype=np.float),grid.lattice.transpose())
            #print('R = ',R)
            for i in np.arange(mol.natoms):
                for j in np.arange(i+1):
                    if j == i:
                        if np.dot(R,R.transpose()) < precision:
                            break
                    Zi=mol.ions[i].Zval
                    Zj=mol.ions[j].Zval
                    rij = mol.ions[i].pos.d_mic(mol.ions[j].pos)-R
                    dij=rij.length()
                    sum += Zi*Zj*sp.erfc(eta*dij)/dij
                    #print(dij,sum)
third=sum
print(third)

142.913040208


# Second term

In [35]:
reciprocal_grid = grid.get_reciprocal()
gg=reciprocal_grid.gg
strf = mol.ions[0].strf(reciprocal_grid) * mol.ions[0].Zval
for i in np.arange(1,mol.natoms):
    strf += mol.ions[i].strf(reciprocal_grid) * mol.ions[i].Zval
strf_sq = strf*np.conjugate(strf)
gg[0,0,0,0]=1
invgg=1.0/gg
invgg[0,0,0,0]=0.0
second=np.real(4.0*np.pi*0.5*np.sum(strf_sq*np.exp(-gg/(4.0*sq_eta))*invgg) / grid.volume)
print(second)

1.04164995209e-53


# Second term (different implementation - not yet started)

In [35]:
reciprocal_grid = grid.get_reciprocal()
gg=reciprocal_grid.gg
strf = mol.ions[0].strf(reciprocal_grid) * mol.ions[0].Zval
for i in np.arange(1,mol.natoms):
    strf += mol.ions[i].strf(reciprocal_grid) * mol.ions[i].Zval
strf_sq = strf*np.conjugate(strf)
gg[0,0,0,0]=1
invgg=1.0/gg
invgg[0,0,0,0]=0.0
second=np.real(4.0*np.pi*0.5*np.sum(strf_sq*np.exp(-gg/(4.0*sq_eta))*invgg) / grid.volume)
print(second)

1.04164995209e-53


# Fourth term

In [36]:
const=-np.sqrt(sq_eta/np.pi)
sum = np.float(0.0)
for i in np.arange(mol.natoms):
    sum+=mol.ions[i].Zval**2
fourth=const*sum 
print(fourth)

-0.507770625193


# Fifth term

In [37]:
const=-4.0*np.pi*0.5*(1.0/(4.0*sq_eta)) / grid.volume
sum = np.float(0.0)
for i in np.arange(mol.natoms):
    sum+=mol.ions[i].Zval
fifth=const*sum
print(fifth)

-16.8689042416


# First term

In [6]:
from pbcpy.hartree import HartreeFunctional
Hartree = HartreeFunctional(rho_of_r)
first=Hartree.energydensity.integral()
print(first)

0.00839883264471


In [39]:
second+third+fourth+fifth

54.079845237404371

In [40]:
np.exp(-np.max(gg[:-3,0,0,0])/(4*sq_eta))*np.max(gg[:,0,0,0]) 

0.0

In [16]:
from pbcpy.local_pseudopotential import NuclearElectron

In [17]:
Nucel = NuclearElectron(mol.ions,rho_of_r,['./Al_lda.oe01.recpot','./Al_lda.oe01.recpot'])

Recpot pseudopotential ./Al_lda.oe01.recpot loaded
Recpot pseudopotential ./Al_lda.oe01.recpot loaded


In [18]:
alpha_Z = -Nucel.energydensity.integral()+2.0*(Hartree.energydensity).integral()

In [19]:
alpha_Z

-1.2701705296858552